<a href="https://colab.research.google.com/github/HiagoAD/GDI2/blob/master/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

## Load datasets


In [47]:
dengue = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2020.csv", encoding ="UTF-8")
emlurb = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2020.csv", encoding ="UTF-8")

dengue19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2019.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb19 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2019.csv", encoding ="UTF-8")

dengue18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2018.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb18 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2018.csv", encoding ="UTF-8")

dengue17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2017.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb17 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2017.csv", encoding ="UTF-8")

dengue16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/dengue2016.csv", encoding ="UTF-8", error_bad_lines=False)
emlurb16 = pd.read_csv("https://raw.githubusercontent.com/HiagoAD/GDI2/master/datasets/emlurb2016.csv", encoding ="UTF-8")


b'Skipping line 108: expected 126 fields, saw 127\nSkipping line 1485: expected 126 fields, saw 127\nSkipping line 2584: expected 126 fields, saw 127\nSkipping line 4415: expected 126 fields, saw 127\nSkipping line 5659: expected 126 fields, saw 127\nSkipping line 6469: expected 126 fields, saw 127\n'
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (55,113) have mixed types.Specify dtype option on import or set low_memory=False.



## Dados de 2020 - 2016


Iremos fazer os mesmos tratamentos anteriores para os dados de 2019 até 2016.

In [48]:
# Dados Dengue

merge_dengue = dengue.append([dengue19,dengue18,dengue17,dengue16], ignore_index = True)
merge_dengue = merge_dengue[["febre", "mialgia",'vomito', "dt_notificacao",'nu_idade','nausea','cefaleia','dor_costas','diabetes','renal']]
merge_dengue['dt_notificacao'] = pd.to_datetime(merge_dengue['dt_notificacao'])

# Agrupamos por número de casos por dia

#merge_dengue = merge_dengue.groupby([pd.Grouper(key="dt_notificacao", freq="D")]).size().reset_index(name="CASOS_DENGUE")


# Dados Emlurb

merge_emlurb = emlurb.append([emlurb19,emlurb18,emlurb17,emlurb16], ignore_index = True)
merge_emlurb = merge_emlurb[["GRUPOSERVICO_CODIGO", "DATA_DEMANDA"]]
merge_emlurb = merge_emlurb[(merge_emlurb.GRUPOSERVICO_CODIGO == 11)]
merge_emlurb["DATA_DEMANDA"] = pd.to_datetime(merge_emlurb["DATA_DEMANDA"],errors='coerce')

# Agrupamos por número de solicitações por dia

#merge_emlurb = merge_emlurb.groupby([pd.Grouper(key="DATA_DEMANDA", freq="D")]).size().reset_index(name="CHAMAD0S_EMLURB")

#year_filter = merge_emlurb[(merge_emlurb['DATA_DEMANDA'].dt.year == 2018)]


merge_dengue = merge_dengue.dropna()
merge_dengue['febre'] = merge_dengue['febre'].astype('int')
merge_dengue['mialgia'] = merge_dengue['mialgia'].astype('int')
merge_dengue['vomito'] = merge_dengue['vomito'].astype('int')
merge_dengue['nausea'] = merge_dengue['nausea'].astype('int')
merge_dengue['cefaleia'] = merge_dengue['cefaleia'].astype('int')
merge_dengue['dor_costas'] = merge_dengue['dor_costas'].astype('int')
merge_dengue['diabetes'] = merge_dengue['diabetes'].astype('int')
merge_dengue['renal'] = merge_dengue['renal'].astype('int')
merge_dengue = merge_dengue.sort_values('dt_notificacao')
merge_dengue

,febre,mialgia,vomito,dt_notificacao,nu_idade,nausea,cefaleia,dor_costas,diabetes,renal
14367,1,1,2,2016-01-03,4024,2,2,2,2,2
13813,2,2,2,2016-01-03,4059,2,2,2,2,2
13658,1,1,2,2016-01-04,4017,1,1,2,2,2
13595,1,1,2,2016-01-04,4031,2,1,2,2,2
13781,1,1,2,2016-01-04,4054,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...
975,1,1,1,2020-07-16,4008,1,1,2,2,2
1007,1,1,2,2020-07-17,4017,2,1,2,2,2
1005,1,2,2,2020-07-19,4010,2,2,2,2,2
1003,1,1,2,2020-07-20,3001,2,2,2,2,2


## Feature de predição

1.   Número de casos de dengue por dia.
2.   Número de solicitações de drenagem por dia.



## Separação de dados: Treino, Validação e Teste

Texto.

In [49]:
def split_values(df, column_name):
  X = df.drop(columns=[column_name,'dt_notificacao']).values
  y = df[column_name].values

  return X, y

In [50]:
features_results = merge_dengue.drop(columns=['febre','dt_notificacao']).values
labels_results = merge_dengue['febre'].values

feature_train, feature_test, label_train, label_test = train_test_split(features_results, labels_results, test_size=0.25, shuffle=False)
feature_train, feature_validation, label_train, label_validation =  train_test_split(feature_train, label_train, test_size=(1/3), shuffle=False)

print(features_results.shape)
print(feature_train.shape)
print(feature_test.shape)
print(feature_validation.shape)

(25913, 8)
(12956, 8)
(6479, 8)
(6478, 8)


In [51]:
# train, validation, test = np.split(merge_dengue.sample(frac=1, random_state=42), [int(.6*len(merge_dengue)), int(.8*len(merge_dengue))])

# column_name = 'febre'

# X_train, y_train = split_values(train, column_name)
# X_valid, y_valid = split_values(validation, column_name)
# X_test,  y_test  = split_values(test, column_name)

X_train, y_train = feature_train, label_train
X_valid, y_valid = feature_validation, label_validation
X_test, y_test = feature_test, label_test

In [52]:
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

(12956, 8)
(6479, 8)
(6478, 8)


## Escolha de Algoritmos

*   Decision Tree
*   Random Forest
*   Naive Bayes
*   MLP
*   KNN



In [53]:
decision_tree = DecisionTreeClassifier()
random_forest = RandomForestClassifier()
naive_bayes   = GaussianNB()
ml_perceptron = MLPClassifier()

decision_tree.fit(X_train, y_train)
y_decision_tree_train = decision_tree.predict(X_train)
y_decision_tree_test  = decision_tree.predict(X_test)

random_forest.fit(X_train, y_train)
y_random_forest_train = random_forest.predict(X_train)
y_random_forest_test  = random_forest.predict(X_test)

naive_bayes.fit(X_train, y_train)
y_naive_bayes_train = naive_bayes.predict(X_train)
y_naive_bayes_test  = naive_bayes.predict(X_test)

ml_perceptron.fit(X_train, y_train)
y_ml_perceptron_train = ml_perceptron.predict(X_train)
y_ml_perceptron_test  = ml_perceptron.predict(X_test)

In [54]:
print("=== Precisão Testes === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_test, y_decision_tree_test))
print("Precisão Random Forest:", accuracy_score(y_test, y_random_forest_test))
print("Precisão Naive Bayes:" , accuracy_score(y_test, y_naive_bayes_test))
print("Precisão MLP: ", accuracy_score(y_test, y_ml_perceptron_test))

print("\n\n=== Precisão Treino === \n\n")

print("Precisão Decision Tree:", accuracy_score(y_train, y_decision_tree_train))
print("Precisão Random Forest:", accuracy_score(y_train, y_random_forest_train))
print("Precisão Naive Bayes:" , accuracy_score(y_train, y_naive_bayes_train))
print("Precisão MLP: ", accuracy_score(y_train, y_ml_perceptron_train))

=== Precisão Testes === 


Precisão Decision Tree: 0.8793023614755363
Precisão Random Forest: 0.8789936718629419
Precisão Naive Bayes: 0.735144312393888
Precisão MLP:  0.8203426454699799


=== Precisão Treino === 


Precisão Decision Tree: 0.8809817845013893
Precisão Random Forest: 0.8809817845013893
Precisão Naive Bayes: 0.7185087990120408
Precisão MLP:  0.8430842852732324


## MLFlow e Hiper-parâmetros

Nessa etapa, adicionaremos o MLFlow e o Optuna para seleção de hiper-parâmetros sobre o conjunto de validação.

In [55]:
!pip install mlflow --quiet
!pip install optuna --quiet

import mlflow
import optuna

### Decision Tree

In [56]:
def objective_decisionTree(trial):
    max_depth = trial.suggest_int("max_depth", 1, 15, 1)
    split     = trial.suggest_int('min_samples_split', 3, 15, 1)
    leaf      = trial.suggest_int('min_samples_leaf', 3, 15, 1)

    decisionTree_selection = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=leaf, min_samples_split=split)
    decisionTree_selection.fit(X_train, y_train)
    
    prediction_validation = decisionTree_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_decisionTree, n_trials=50)
print(study_dt.best_trial)

[I 2021-08-15 16:59:21,230] A new study created in memory with name: no-name-eac3595f-57eb-46af-80cf-3c645017aa60
[I 2021-08-15 16:59:21,250] Trial 0 finished with value: 0.8965730163630751 and parameters: {'max_depth': 12, 'min_samples_split': 15, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8965730163630751.
[I 2021-08-15 16:59:21,260] Trial 1 finished with value: 0.8484100030873727 and parameters: {'max_depth': 2, 'min_samples_split': 15, 'min_samples_leaf': 13}. Best is trial 0 with value: 0.8965730163630751.
[I 2021-08-15 16:59:21,272] Trial 2 finished with value: 0.891015745600494 and parameters: {'max_depth': 4, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 0 with value: 0.8965730163630751.
[I 2021-08-15 16:59:21,286] Trial 3 finished with value: 0.8971904908922507 and parameters: {'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 6}. Best is trial 3 with value: 0.8971904908922507.
[I 2021-08-15 16:59:21,302] Trial 4 finished with value: 0.896

FrozenTrial(number=3, values=[0.8971904908922507], datetime_start=datetime.datetime(2021, 8, 15, 16, 59, 21, 274140), datetime_complete=datetime.datetime(2021, 8, 15, 16, 59, 21, 286586), params={'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 6}, distributions={'max_depth': IntUniformDistribution(high=15, low=1, step=1), 'min_samples_split': IntUniformDistribution(high=15, low=3, step=1), 'min_samples_leaf': IntUniformDistribution(high=15, low=3, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None)


In [57]:
print(study_dt.best_trial.value)
print(study_dt.best_trial.params)

0.8971904908922507
{'max_depth': 7, 'min_samples_split': 12, 'min_samples_leaf': 6}


### Random Forest

In [58]:
def objective_randomForest(trial):
    forrest_n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    forrest_criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    forrest_max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])

    randomForrest_selection = RandomForestClassifier(n_estimators=forrest_n_estimators, criterion=forrest_criterion, max_features=forrest_max_features)
    randomForrest_selection.fit(X_train, y_train) 

    prediction_validation = randomForrest_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_randomForest, n_trials=50)
print(study_rf.best_trial)

[I 2021-08-15 16:59:22,281] A new study created in memory with name: no-name-5ac92943-f82a-4e77-a492-a2cf2db32531
[I 2021-08-15 16:59:26,632] Trial 0 finished with value: 0.8920963260265514 and parameters: {'n_estimators': 513, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8920963260265514.
[I 2021-08-15 16:59:31,120] Trial 1 finished with value: 0.8914788514973757 and parameters: {'n_estimators': 544, 'criterion': 'gini', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8920963260265514.
[I 2021-08-15 16:59:38,899] Trial 2 finished with value: 0.8911701142327879 and parameters: {'n_estimators': 830, 'criterion': 'entropy', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8920963260265514.
[I 2021-08-15 16:59:42,607] Trial 3 finished with value: 0.8919419573942575 and parameters: {'n_estimators': 408, 'criterion': 'entropy', 'max_features': 'sqrt'}. Best is trial 0 with value: 0.8920963260265514.
[I 2021-08-15 16:59:44,591] Trial 4 finished wit

FrozenTrial(number=0, values=[0.8920963260265514], datetime_start=datetime.datetime(2021, 8, 15, 16, 59, 22, 284912), datetime_complete=datetime.datetime(2021, 8, 15, 16, 59, 26, 632351), params={'n_estimators': 513, 'criterion': 'gini', 'max_features': 'sqrt'}, distributions={'n_estimators': IntUniformDistribution(high=1000, low=100, step=1), 'criterion': CategoricalDistribution(choices=('gini', 'entropy')), 'max_features': CategoricalDistribution(choices=('auto', 'sqrt', 'log2'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)


In [59]:
print(study_rf.best_trial.value)
print(study_rf.best_trial.params)

0.8920963260265514
{'n_estimators': 513, 'criterion': 'gini', 'max_features': 'sqrt'}


### Naive Bayes

In [ ]:
def objective_naiveBayes(trial):
    nb_var_smoothing = trial.suggest_float('var_smoothing', 1e-9, 1e9, log=True)

    naiveBayes_selection = GaussianNB(var_smoothing=nb_var_smoothing)
    naiveBayes_selection.fit(X_train, y_train)     

    prediction_validation = naiveBayes_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_nb = optuna.create_study(direction="maximize")
study_nb.optimize(objective_naiveBayes, n_trials=50)
print(study_nb.best_trial)

In [61]:
print(study_nb.best_trial.value)
print(study_nb.best_trial.params)

0.8934856437171966
{'var_smoothing': 5.544667198090331e-06}


### MLP

In [116]:
def objective_MLP(trial):
    layer_size = trial.suggest_int('hidden_layer_sizes', 10, 1000, 10)
    activation = trial.suggest_categorical('activation', ['logistic', 'relu', 'tanh'])
    solver = trial.suggest_categorical('solver', ['adam', 'sgd'])
    mlp_selection = MLPClassifier(layer_size, activation=activation, solver=solver)
    mlp_selection.fit(X_train, y_train)     

    prediction_validation = mlp_selection.predict(X_valid)
    accuracy_validation = accuracy_score(y_valid, prediction_validation)

    return accuracy_validation

study_mlp = optuna.create_study(direction="maximize")
study_mlp.optimize(objective_MLP, n_trials=50)
print(study_mlp.best_trial)

[I 2021-08-15 18:42:35,195] A new study created in memory with name: no-name-dc5c6010-eb9f-4da4-94f4-31cf0c07be59
[I 2021-08-15 18:42:41,212] Trial 0 finished with value: 0.8471750540290213 and parameters: {'hidden_layer_sizes': 230, 'activation': 'relu', 'solver': 'adam'}. Best is trial 0 with value: 0.8471750540290213.
[I 2021-08-15 18:42:46,035] Trial 1 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 430, 'activation': 'logistic', 'solver': 'sgd'}. Best is trial 1 with value: 0.8672429762272307.
[I 2021-08-15 18:42:53,116] Trial 2 finished with value: 0.8511886384686632 and parameters: {'hidden_layer_sizes': 340, 'activation': 'relu', 'solver': 'adam'}. Best is trial 1 with value: 0.8672429762272307.
[I 2021-08-15 18:42:55,892] Trial 3 finished with value: 0.8672429762272307 and parameters: {'hidden_layer_sizes': 180, 'activation': 'logistic', 'solver': 'adam'}. Best is trial 1 with value: 0.8672429762272307.
[I 2021-08-15 18:43:05,189] Trial 4 finishe

FrozenTrial(number=47, values=[0.8726458783575177], datetime_start=datetime.datetime(2021, 8, 15, 18, 46, 29, 522399), datetime_complete=datetime.datetime(2021, 8, 15, 18, 46, 43, 748545), params={'hidden_layer_sizes': 730, 'activation': 'relu', 'solver': 'adam'}, distributions={'hidden_layer_sizes': IntUniformDistribution(high=1000, low=10, step=10), 'activation': CategoricalDistribution(choices=('logistic', 'relu', 'tanh')), 'solver': CategoricalDistribution(choices=('adam', 'sgd'))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=47, state=TrialState.COMPLETE, value=None)


In [117]:
print(study_mlp.best_trial.value)
print(study_mlp.best_trial.params)

0.8726458783575177
{'hidden_layer_sizes': 730, 'activation': 'relu', 'solver': 'adam'}


## Diagnóstico de melhor modelo

Após os testes realizados com o Random Forest, Decision Tree, Naive Bayes e MLP, encontramos que o algoritmo que melhor resultado obeteve foi blabla.


In [ ]:
## heatmap

## Feature importance

Verificamos o nível de relevância das colunas do dataset.

# **MLP**

In [64]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers, optimizers, metrics, activations
from keras.callbacks import EarlyStopping

resolver = tensorflow.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tensorflow.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tensorflow.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tensorflow.config.list_logical_devices('TPU'))

INFO:tensorflow:Initializing the TPU system: grpc://10.81.251.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.81.251.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


All devices:  [LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:7', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:6', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:5', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:4', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:3', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:0', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:1', device_type='TPU'), LogicalDevice(name='/job:worker/replica:0/task:0/device:TPU:2', device_type='TPU')]


In [108]:
def MLP(input_shape):
    curve = activations.sigmoid
    model = keras.Sequential(
        [
            layers.InputLayer(input_shape),
            layers.Dense(input_shape, activation=curve),
            layers.Dense(input_shape, activation=curve),
            layers.Dropout(0.1),
            layers.Dense(input_shape, activation=curve),
            layers.Dense(1, activation=activations.softmax)
        ]
    )
    model.compile(loss="mean_squared_error", optimizer=optimizers.SGD(learning_rate=1), metrics=[metrics.Accuracy(), metrics.Recall(), metrics.Precision()], steps_per_execution=50)

    return model

In [109]:
mlp = MLP(X_train.shape[1])
mlp.fit(X_train, y_train, batch_size=100, epochs=1000, callbacks=[EarlyStopping(patience=10)], validation_data=(X_valid, y_valid))
mlp.evaluate(X_test, y_test, batch_size=100)

Epoch 1/1000
130/130 [==============================] - 2s 17ms/step - loss: 0.2438 - accuracy: 0.7562 - recall_4: 1.0000 - precision_4: 1.0000 - val_loss: 0.1328 - val_accuracy: 0.8672 - val_recall_4: 1.0000 - val_precision_4: 1.0000
Epoch 2/1000
130/130 [==============================] - 0s 939us/step - loss: 0.2438 - accuracy: 0.7562 - recall_4: 1.0000 - precision_4: 1.0000 - val_loss: 0.1328 - val_accuracy: 0.8672 - val_recall_4: 1.0000 - val_precision_4: 1.0000
Epoch 3/1000
130/130 [==============================] - 0s 977us/step - loss: 0.2438 - accuracy: 0.7562 - recall_4: 1.0000 - precision_4: 1.0000 - val_loss: 0.1328 - val_accuracy: 0.8672 - val_recall_4: 1.0000 - val_precision_4: 1.0000
Epoch 4/1000
130/130 [==============================] - 0s 961us/step - loss: 0.2438 - accuracy: 0.7562 - recall_4: 1.0000 - precision_4: 1.0000 - val_loss: 0.1328 - val_accuracy: 0.8672 - val_recall_4: 1.0000 - val_precision_4: 1.0000
Epoch 5/1000
130/130 [==============================] - 0

[0.09816329926252365, 0.9018366932868958, 1.0, 1.0]